In [1]:
#loading ema workbench and packages

import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import csv

from __future__ import (division, unicode_literals, print_function, absolute_import)

from ema_workbench import (Model, RealParameter, TimeSeriesOutcome, ScalarOutcome, 
                           perform_experiments, ema_logging, Constant, load_results, 
#                            MultiprocessingEvaluator, IpyparallelEvaluator
                          )
from ema_workbench.em_framework.parameters import Policy, create_parameters
from ema_workbench.connectors.vensim import VensimModel, LookupUncertainty, VensimModelStructureInterface
from ema_workbench.em_framework.evaluators import (LHS, SOBOL, MORRIS, MC)
from ema_workbench.util import save_results, ema_logging, CaseError
from ema_workbench.analysis.plotting import lines, envelopes
from ema_workbench.analysis.plotting_util import KDE, BOXPLOT
from ema_workbench.analysis.pairs_plotting import pairs_lines, pairs_scatter
from ema_workbench.em_framework.parameters import Policy
from ema_workbench.em_framework import CategoricalParameter

ema_logging.log_to_stderr(ema_logging.INFO)



C:\Users\martijn.legene\AppData\Local\Continuum\anaconda3\lib\site-packages\ema_workbench\em_framework\optimization.py:22: ImportWarning: platypus based optimization not available
  warnings.warn("platypus based optimization not available", ImportWarning)
C:\Users\martijn.legene\AppData\Local\Continuum\anaconda3\lib\site-packages\ema_workbench\connectors\__init__.py:18: ImportWarning: netlogo connector not available
  warnings.warn("netlogo connector not available", ImportWarning)
C:\Users\martijn.legene\AppData\Local\Continuum\anaconda3\lib\site-packages\ema_workbench\connectors\__init__.py:23: ImportWarning: pysd connector not available
  warnings.warn("pysd connector not available", ImportWarning)


<Logger EMA (DEBUG)>

In [2]:
#load vensim model
mdl_file = 'Accessibility model views v209.vpm'
model = VensimModel("defaultModel",model_file=mdl_file)

# define common uncertainties:
model.uncertainties = [
#AV variables  
    #Land use    
    #Traffic    
                        CategoricalParameter('Switch Penetration rate AV', (0, 1, 2, 3, 4)), #4 for no AVs at all, so base scenario
                        RealParameter('Value of time in private AV', 5.39, 10.84),
                        RealParameter('Empty vehicle trips to avoid parking', 0, 0.2), #12 minutes parking searchtime = 0.2 hour
                        RealParameter('Maximum increased efficiency by AV', 1, 1.4), # PCU of 0.7 = 1.4
    #Parking
                        RealParameter('Increased parking density rate', 1, 1.6),
#                        RealParameter('Idle time car', 5.5, 84),
#                        RealParameter('Fraction new uses for cars', -0.1, 0.5),
                        RealParameter('New uses for cars', 0.16, 0.945),
#Decreasing the idle time of a car from 84% or 5.5%, so use the car 1-new uses for cars. car is used 16% of the time to 94.5%
                        RealParameter('Fraction mobility for those unable to drive', 0, 0.5),
                        RealParameter('Carsharing rate', 0, 0.99), 
    #Population
    #Modal split

#Policy variables
    #Land use
    
##                         RealParameter('Policy allocation priority[Housing]', 0, 0.589),
##                         RealParameter('Policy allocation priority[Business]', 0, 0.896),
##                         RealParameter('Policy allocation priority[Road]', 0, 0.1),
##                         RealParameter('Policy allocation priority[Parking]', 0, 0.1),
##                         RealParameter('Policy allocation priority[Other]', 0, 1),
    
#                        RealParameter('Build higher', 0, 0.5),
#                        RealParameter('Create free space', 0, 0.5),
#                        RealParameter('Dynamic urban space factor', 1, 1.5),
    
##                         RealParameter('Vacancy threshold', 0.05, 0.15),
##                         RealParameter('Delay time housing demolition', 1, 10),
    
    #Traffic
##                         RealParameter('Threshold value for new lane construction', 0.1, 0.4),
##                         RealParameter('Threshold value for lane decommissioning', 1.05, 1.3),
##                         RealParameter('Delay time lane decommissioning', 1, 10),
                        
    #Parking
##                         RealParameter('Threshold value for parking decommissioning', 0.05, 0.15),
##                         RealParameter('Delay time parking decommissioning', 1, 3),
##                         RealParameter('Parking price multiplier', 0, 2),
    #Population
    #Modal split
##                         RealParameter('Attention for car traffic management', 0, 0.5),
##                         RealParameter('Attention for AV traffic management', 0, 1),
##                         RealParameter('Attention for PT', 0, 1),
##                         RealParameter('Attention for active modes', 0, 1),

#Global uncertainties
    #Land use
                        RealParameter('Initial vacancy rate city', 0.02, 0.06),
                        RealParameter('Economic growth rate', 0, 0.03),
#                         RealParameter('Building time', 1, 2),
#                         RealParameter('Construction aging', 60, 100),
#     #Traffic
#                         RealParameter('Construction time new road lanes', 2.5, 7.5),
# #                        RealParameter('Road decay', 15, 25),
#     #Parking
#                         RealParameter('Construction time parkings', 0.25, 0.75),  
#                         RealParameter('Parking decay', 40, 60),
    #Population
                        RealParameter('Migration rate', 0, 0.0042),
                        RealParameter('Average time in house', 5, 40)]
    #Modal split 
#                        RealParameter('Value of time car passenger', 6, 12)

                        
                        
#model runt goed met de outcomes voor 1 zone        
# scenarios  #use CategoricalParameter to use switches in the model
                        
                            # CategoricalParameter('Switch Building Mode', (0,1)),
                        
                           
model.outcomes = [
                    TimeSeriesOutcome('TIME'),
#Land use    
#                     TimeSeriesOutcome('Jobs per district type[City center]'),
#                     TimeSeriesOutcome('Jobs per district type[Other urban districts]'),
#                     TimeSeriesOutcome('Jobs per district type[Suburbs]'),
#                     TimeSeriesOutcome('Jobs per district type[Rural areas]'),                
#                     TimeSeriesOutcome('Jobs density per district type[City center]'),
#                     TimeSeriesOutcome('Jobs density per district type[Other urban districts]'),
#                     TimeSeriesOutcome('Jobs density per district type[Suburbs]'),
#                     TimeSeriesOutcome('Jobs density per district type[Rural areas]'),
                    TimeSeriesOutcome('Fraction housing per district type[City center]'),    
                    TimeSeriesOutcome('Fraction housing per district type[Other urban districts]'),
                    TimeSeriesOutcome('Fraction housing per district type[Suburbs]'),
#                    TimeSeriesOutcome('Fraction housing per district type[Rural areas]'),
                    TimeSeriesOutcome('Fraction business per district type[City center]'),    
                    TimeSeriesOutcome('Fraction business per district type[Other urban districts]'),
                    TimeSeriesOutcome('Fraction business per district type[Suburbs]'),
#                    TimeSeriesOutcome('Fraction business per district type[Rural areas]'),
                    TimeSeriesOutcome('Fraction road per district type[City center]'),    
                    TimeSeriesOutcome('Fraction road per district type[Other urban districts]'),
                    TimeSeriesOutcome('Fraction road per district type[Suburbs]'),
#                    TimeSeriesOutcome('Fraction road per district type[Rural areas]'),
                    TimeSeriesOutcome('Fraction parking per district type[City center]'),    
                    TimeSeriesOutcome('Fraction parking per district type[Other urban districts]'),
#parking places in suburbs and rural areas are not taken into account
#                     TimeSeriesOutcome('Fraction parking per district type[Suburbs]'),
#                     TimeSeriesOutcome('Fraction parking per district type[Rural areas]'),
                    TimeSeriesOutcome('Fraction other land per district type[City center]'),    
                    TimeSeriesOutcome('Fraction other land per district type[Other urban districts]'),
                    TimeSeriesOutcome('Fraction other land per district type[Suburbs]'),
#                    TimeSeriesOutcome('Fraction other land per district type[Rural areas]'),
    
#Traffic    
                    TimeSeriesOutcome('Distance within acceptable travel time per district type[City center]'),
                    TimeSeriesOutcome('Distance within acceptable travel time per district type[Other urban districts]'),
                    TimeSeriesOutcome('Distance within acceptable travel time per district type[Suburbs]'),
#                    TimeSeriesOutcome('Distance within acceptable travel time per district type[Rural areas]'),
                    TimeSeriesOutcome('Total traffic volume in morning peak per district type[City center]'),
                    TimeSeriesOutcome('Total traffic volume in morning peak per district type[Other urban districts]'),
                    TimeSeriesOutcome('Total traffic volume in morning peak per district type[Suburbs]'),
#                    TimeSeriesOutcome('Total traffic volume in morning peak per district type[Rural areas]'),
                    TimeSeriesOutcome('Actual road capacity use compared to initial optimal per district type[City center]'),
                    TimeSeriesOutcome('Actual road capacity use compared to initial optimal per district type[Other urban districts]'),
                    TimeSeriesOutcome('Actual road capacity use compared to initial optimal per district type[Suburbs]'),
#                    TimeSeriesOutcome('Actual road capacity use compared to initial optimal per district type[Rural areas]'),
#Parking    
#                  TimeSeriesOutcome('Parking demand per district type[City center]'),
#                   TimeSeriesOutcome('Parking demand per district type[Other urban districts]'),
#parking places in suburbs and rural areas are not taken into account
#                     TimeSeriesOutcome('Parking demand per district type[Suburbs]'),
#                     TimeSeriesOutcome('Parking demand per district type[Rural areas]'),
                    TimeSeriesOutcome('Parking places per district type[City center]'),
                    TimeSeriesOutcome('Parking places per district type[Other urban districts]'),
                    
#parking places in suburbs and rural areas are not taken into account
#                     TimeSeriesOutcome('Parking places per district type[Suburbs]'),                
#                     TimeSeriesOutcome('Parking places per district type[Rural areas]'),
#Population    
#                     TimeSeriesOutcome('Attractiveness to live in district type[City center]'),
#                     TimeSeriesOutcome('Attractiveness to live in district type[Other urban districts]'),
#                     TimeSeriesOutcome('Attractiveness to live in district type[Suburbs]'),
#                     TimeSeriesOutcome('Attractiveness to live in district type[Rural areas]'),
                    TimeSeriesOutcome('Accessibility to jobs per district type[City center]'),
                    TimeSeriesOutcome('Accessibility to jobs per district type[Other urban districts]'),
                    TimeSeriesOutcome('Accessibility to jobs per district type[Suburbs]'),
#                    TimeSeriesOutcome('Accessibility to jobs per district type[Rural areas]'),
#                    TimeSeriesOutcome('Total population in GCA'),
                    TimeSeriesOutcome('Population per district[Amager]'),
                    TimeSeriesOutcome('Population per district[Christianshavn]'),
                    TimeSeriesOutcome('Population per district[Eastern Zealand]'),
                    TimeSeriesOutcome('Population per district[Frederiksberg]'),
                    TimeSeriesOutcome('Population per district[Indre By]'),
                    TimeSeriesOutcome('Population per district[Norrebro]'),
                    TimeSeriesOutcome('Population per district[Northern Suburbs]'),
                    TimeSeriesOutcome('Population per district[Northern Zealand]'),
                    TimeSeriesOutcome('Population per district[Osterbro]'),
                    TimeSeriesOutcome('Population per district[Vestegnen]'),
                    TimeSeriesOutcome('Population per district[Vesterbro]'),
                    TimeSeriesOutcome('Population density per district type[City center]'),
                    TimeSeriesOutcome('Population density per district type[Other urban districts]'),
                    TimeSeriesOutcome('Population density per district type[Suburbs]'),
                    TimeSeriesOutcome('Population density per district type[Rural areas]'),
#Modal split    
                    TimeSeriesOutcome('GCA Modal split car and AV')
                ]
    
#print results
 
#perform experiments
#nr_experiments = 1000
#perform more experiments when the script is finished
    
#results = perform_experiments(model, nr_experiments)

In [5]:
#perform experiments
nr_experiments = 5000

    
results5000 = perform_experiments(model, nr_experiments, reporting_interval=100)

#save results
fn = r'PRIM results/no_policy_experiments_PRIM_{}.tar.gz'.format(nr_experiments)
save_results(results5000, fn)

[MainProcess/INFO] performing 5000 scenarios * 1 policies * 1 model(s) = 5000 experiments
[MainProcess/INFO] performing experiments sequentially
[MainProcess/INFO] 100 cases completed
[MainProcess/INFO] 200 cases completed
[MainProcess/INFO] 300 cases completed
[MainProcess/INFO] 400 cases completed
[MainProcess/INFO] 500 cases completed
[MainProcess/INFO] 600 cases completed
[MainProcess/INFO] 700 cases completed
[MainProcess/INFO] 800 cases completed
[MainProcess/INFO] 900 cases completed
[MainProcess/INFO] 1000 cases completed
[MainProcess/INFO] 1100 cases completed
[MainProcess/INFO] 1200 cases completed
[MainProcess/INFO] 1300 cases completed
[MainProcess/INFO] 1400 cases completed
[MainProcess/INFO] 1500 cases completed
[MainProcess/INFO] 1600 cases completed
[MainProcess/INFO] 1700 cases completed
[MainProcess/INFO] 1800 cases completed
[MainProcess/INFO] 1900 cases completed
[MainProcess/INFO] 2000 cases completed
[MainProcess/INFO] 2100 cases completed
[MainProcess/INFO] 2200 

In [ ]:
results5000 = load_results(r'PRIM results/experiments_PRIM_5000.tar.gz')

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from __future__ import (division, print_function)

from ema_workbench.analysis import prim
from ema_workbench import ema_logging, load_results

ema_logging.log_to_stderr(level=ema_logging.INFO)

def classify(outcomes):
    #get the output for variables you want
    resultmodalsplit = outcomes['GCA Modal split car and AV']
#    resultpopdenscitycenter = outcomes['Population density per district type[City center]']
#    resultparkingcitycenter = outcomes['Fraction parking per district type[City center]']
    
    #make an empty array of length equal to number of cases 
    classesmodalsplit =  np.zeros(resultmodalsplit.shape[0])
#    classespopdenscitycenter =  np.zeros(resultpopdenscitycenter.shape[0])
#    classesparkingcitycenter =  np.zeros(resultparkingcitycenter.shape[0])
    
    #if GCA Modal split car and AV is higher than 0.4, classify as 1 
    classesmodalsplit[resultmodalsplit[:, -1] > 0.4] = 1
    #if Population density per district type[City center] is lower than initial value of 5653 p/km2, classify as 1 
#    classespopdenscitycenter[resultpopdenscitycenter[:, -1] < 5653] = 1
    #if Fraction parking per district type[City center] is higher than initial value of 0.1077, classify as 1 
#    classesparkingcitycenter[resultparkingcitycenter[:, -1] > 0.1077] = 1
    
    return classesmodalsplit
#    return classespopdenscitycenter
#    return classesparkingcitycenter

#load data
#results = load_results(r'PRIM results/experiments_PRIM_10.tar.gz'+r'PRIM results/experiments_PRIM_20.tar.gz'+r'PRIM results/experiments_PRIM_30.tar.gz')
#experiments, results = results
results = results5000

In [ ]:
import ema_workbench.analysis.pairs_plotting as pairs
#perform prim on modified results tuple
prim_obj = prim.setup_prim(results, classify, threshold=0.8, threshold_type=1)

box_1 = prim_obj.find_box()
box_1.show_ppt()
box_1.show_tradeoff()
box_1.inspect( style='graph', boxlim_formatter="{: .2f}", 
               ticklabel_formatter="{} {: .2f}")
box_1.inspect()
box_1.select(8)
box_1.write_ppt_to_stdout()
#box_1.show_pairs_scatter() #TypeError: float() argument must be a string or a number, not 'set'

#print prim to std_out
print(prim_obj.stats_to_dataframe())
print(prim_obj.boxes_to_dataframe())

#visualize
prim_obj.display_boxes()
plt.show()

In [ ]:
prim_obj = prim.setup_prim(results, classify, mass_min=0.05, threshold=0.8)

# let's find a first box
box1 = prim_obj.find_box()

# let's analyze the peeling trajectory
box1.show_ppt()
box1.show_tradeoff()

box1.write_ppt_to_stdout()

# based on the peeling trajectory, we pick entry number 3
box1.select(22)

# show the resulting box
prim_obj.display_boxes()
#prim_obj.write_boxes_to_stdout() #AttributeError: 'Prim' object has no attribute 'write_boxes_to_stdout'

plt.show()

In [ ]:
#extract results for 1 policy
# logicalIndex = experiments['policy'] == 'no policy'
# newExperiments = experiments[ logicalIndex ]
# newResults = {}
# for key, value in results.items():
#     newResults[key] = value[logicalIndex]

# results = (newExperiments, newResults)

#perform prim on modified results tuple
boxes = prim.setup_prim(results, classify, 
                                    threshold=0.8, 
                                    threshold_type=1)

#print prim to std_out
box1.write_ppt_to_stdout()# AttributeError: module 'ema_workbench.analysis.prim' has no attribute 'write_prim_to_stdout

#visualize
box1.show_boxes_individually(boxes, results)
prim.show_boxes_together(boxes, results)
plt.show()